# santander-customer-satisfaction

# import

In [ ]:
import numpy as np
import pandas as pd
import tqdm
import os
from collections import deque
from collections import Counter
import codecs
import copy
from sklearn.metrics import confusion_matrix
import sys
from platform import python_version
from datetime import datetime
import time
import shutil
from sklearn.preprocessing import OneHotEncoder
import csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression



from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE


# environment

In [ ]:
print(python_version())

In [ ]:
sys.getdefaultencoding()

In [ ]:
%autosave 0

In [ ]:
#最大表示列数の指定（ここでは2000列を指定）
pd.set_option('display.max_columns', 2000)

In [ ]:
#最大表示行数の指定（ここでは200行を指定）
pd.set_option('display.max_rows', 50)

# define

In [ ]:
# n_estimators:ここを増やすと精度は上がるが遅い。。。
ESTIMATOR_NUM=10
RANDOM_STATE_VAL = 0
target_col_name = "TARGET"
id_col_name = "ID"
CV_VAL = 5

#FEATURE_NUM = 950
#FEATURE_NUM = 150
FEATURE_NUM = 200

SELECTOR_STEP = .05

# function

In [ ]:
def dispalyDf(readDf, dispFlag, row_num, dispLabel):
    print("========================= {} =========================".format(dispLabel))
    row_num , col_num = np.shape(readDf)
    print("row_num = {}, col_num = {}".format(row_num, col_num))
    if dispFlag:
        display(readDf.head(row_num))
    print("++++++ None Count All++++++")
    print(readDf.isnull().values.sum())
    print("++++++ None Count ++++++")
    print(readDf.isnull().sum())
    print("++++++ Col Type ++++++")
    print(readDf.dtypes)
#     print("++++++ Unique By Col（Noneは含まないので注意） ++++++")
#     for col_name in readDf.columns.tolist():
#         print("{} : {}".format(col_name, readDf[col_name].nunique()))

In [ ]:
################################################
######## チェック関数
################################################
def chkVal(expect_label, expect_num, act_label, act_num):
    if expect_num != act_num:
        raise Exception("Num Error !! expect({}): {}, act({}) : {}"
                        .format(expect_label, expect_num, act_label, act_num))


In [ ]:
def getStandardScalerDf(inputDf):
    row_num, col_num = np.shape(inputDf)
    
    # スケール変換
    scaler = StandardScaler()
    
    val_array = scaler.fit_transform(np.array(inputDf.values.tolist()))
    standardScDf = pd.DataFrame(val_array, dtype=np.float, columns = inputDf.columns.tolist())
    
    sc_row_num, sc_col_num = np.shape(standardScDf)

    # 行数チェック
    chkVal("expect_row_num", row_num, "sc_row_num", sc_row_num)
    # 列数チェック
    chkVal("expect_col_num", col_num, "sc_col_num", sc_col_num)

    
    del val_array
    return standardScDf


In [ ]:
# test用のカラムのDataFrame
def getAdjustTestColDf(inputDf, train_col_list):
    not_enough_col_list = set(train_col_list) - set(inputDf.columns.tolist())
    print("# trainに有ってtestにないカラム名のリスト")
    print("{} : {}".format(len(not_enough_col_list), not_enough_col_list))
        
    dataLen = len(inputDf)
        
    retDf = inputDf.join(
        pd.DataFrame({col_name : [0]*dataLen  for col_name  in not_enough_col_list },
                     dtype=np.float, columns=not_enough_col_list)
    ).loc[:, train_col_list]
    
    # カラムリストが一致するかチェック
    if retDf.columns.tolist() != train_col_list:
        raise Exception("col_name_list not Match train and test.")

    # データ行数が一致するかチェック
    chkVal("dataLen", dataLen, "retDf", len(retDf))

    return retDf

# process

In [ ]:
act_train_read_Df = pd.read_csv('input/train.csv')
act_test_read_Df = pd.read_csv('input/test.csv')

In [ ]:
act_train_Df = act_train_read_Df.copy()
act_test_Df = act_test_read_Df.copy()

In [ ]:
dispalyDf(act_train_Df, True, 10, "act_train_Df")

In [ ]:
dispalyDf(act_test_Df, True, 10, "act_test_Df")

In [ ]:
################################################

In [ ]:
# train:データのDataFrameと、ターゲットのデータフレームに分ける。
train_target_Ser = act_train_Df[target_col_name]
act_train_Df = act_train_Df.loc[:,[col_name for col_name in act_train_Df.columns.tolist() if col_name not in [id_col_name,target_col_name]]]
print(set(train_target_Ser))

In [ ]:
# test:idとデータのDataFrameを抽出。
test_id_Df = act_test_Df.loc[:,[id_col_name]]
act_test_Df = act_test_Df.loc[:,[col_name for col_name in act_test_Df.columns.tolist() if col_name != id_col_name]]

In [ ]:
drop_columns = []
train_val_range_dict = {}

In [ ]:
def getDropListEqualVal(input_Df, drop_columns):
    col_list_1st = input_Df.columns.tolist()
    col_list_2nd = col_list_1st.copy()
    
    for col_name1 in col_list_1st:
        col_list_2nd.remove(col_name1)
        
        drop_columns.extend([col_name2 for col_name2 in col_list_2nd
            if col_name2 not in drop_columns and (input_Df[col_name1] == input_Df[col_name2]).all()
        ])
    
    return drop_columns

In [ ]:
def getSerTestValInRange(inputSer, val_range_list):
    train_min_val = val_range_list[0]
    train_max_val = val_range_list[1]
    return np.where(inputSer<train_min_val,train_min_val, 
                         np.where(inputSer>train_max_val, train_max_val, inputSer))    

In [ ]:
###### 前処理 ######
def getBeroreProcessDf(input_Df, drop_columns, trainFlag, train_val_range_dict):
    retDf = input_Df.copy()

    #### 訓練時のみの処理
    if trainFlag:
        # 値のユニーク数が１のカラムをdropリストに追加する
        drop_columns.extend([col_name for col_name in act_train_Df.columns.tolist()
                            if act_train_Df[col_name].nunique() == 1 not in drop_columns])
        
        # 値が同じカラムをdropリストに追加する。
        drop_columns = getDropListEqualVal(input_Df, drop_columns)
    ####
    
    retDf = retDf.drop(drop_columns, axis=1)
    
    columns = retDf.columns.tolist()


    for col_name in columns:
        if retDf[col_name].dtype == object:
#             # 文字列のカラムの加工処理
#             retDf[col_name] = retDf[col_name].fillna('type 0')
#             retDf[col_name] = retDf[col_name].apply(lambda x: x.split(' ')[1])
            
#         # int64に変換する。
#         retDf[col_name] = pd.to_numeric(retDf[col_name])
            pass
        elif retDf[col_name].dtype == np.int64:
            # float64に変換する。
            retDf[col_name] = retDf[col_name].astype(np.float64)

            
        if trainFlag:
            # train時
            # 最小と最大を取得
            train_val_range_dict[col_name] = [
                retDf[col_name].min(), retDf[col_name].max()
            ]
        else:
            # test時
            # 最小〜最大の範囲内に収める
            retDf[col_name]  = getSerTestValInRange(retDf[col_name] , train_val_range_dict[col_name])
        
     
    # 桁数が大きい値をlogにする
    retDf["var38"] = np.log(retDf["var38"])
    
    return retDf, drop_columns, train_val_range_dict
    

In [ ]:
act_train_Df_2, drop_columns, train_val_range_dict = getBeroreProcessDf(act_train_Df, drop_columns, True, train_val_range_dict)
# デバッグ用
#act_train_Df_2, drop_columns, train_val_range_dict = getBeroreProcessDf(act_train_Df[:1000], drop_columns, True, train_val_range_dict)

In [ ]:
dispalyDf(act_train_Df_2, True, 10, "act_train_Df_2")

In [ ]:
act_test_Df_2, _, _ = getBeroreProcessDf(act_test_Df, drop_columns, False, train_val_range_dict)

In [ ]:
dispalyDf(act_test_Df_2, True, 10, "act_test_Df_2")

In [ ]:
####################################
##### 次元削減(特徴量選択) を使用する
##### ↓特徴量選択
####################################
def getSelectorFeature(inputFeatureDf,y_Df,feature_num, selector_step, model):

    print("特徴量選択数 = ",feature_num)
    if(feature_num < 0):
        # デフォルト値
        print("n_features_to_select = None に設定")
        feature_num = None
    elif feature_num > len(inputFeatureDf.columns.tolist()):
        print ("feature_num({}) is Over colnum({})".format(feature_num, len(inputFeatureDf.columns.tolist())))
        # 次元削減せずに返却する
        return inputFeatureDf
    
    # 特徴量選択
    # step:特徴量削除の速度。一度の再帰処理により指定ステップ分の特徴量が消滅する。
#     selector = RFE(estimator=LogisticRegression(penalty='l2', C=1.5, random_state=RANDOM_STATE_VAL, max_iter=120,
#                                            solver='liblinear'),
    selector = RFE(estimator=model,
                   n_features_to_select=feature_num,
                   step=selector_step)
    selector.fit(inputFeatureDf,y_Df.values.ravel())
    #selector.fit(inputFeatureDf, y_Df.iloc[:,0])

    # 削除対象配列(Falseが削除対象)
    selFlagArray = selector.support_
    
    # 現状のカラム名を取得
    columnIndex = inputFeatureDf.columns
    
    # 削除対象カラム名リスト
    drop_column_list = list()
    # 削除対象配列 でループ(i=0,1,2,・・・)
    #print(len(selFlagArray))
    for i in range(len(selFlagArray)):
        # Falseの場合に、削除対象リストに追加
        if(selFlagArray[i] == False):
            drop_column_list.append(columnIndex[i])

    # 不要な列削除
    #print("削除対象カラム = " , drop_column_list)
    inputFeatureNewDf = inputFeatureDf.drop(drop_column_list, axis=1)
    

    return inputFeatureNewDf


In [ ]:
# Pipelineに使う独自関数
class MyStandardScaler(object):
    def fit(self, X, y):
        print(" MyStandardScaler.fit")
        return X
    def fit_transform(self, X, y):
        print(" MyStandardScaler.fit_transform")
        return getStandardScalerDf(X)
#    def transform(self, X, y):
    def transform(self, X):
        print(" MyStandardScaler.transform")
        return getStandardScalerDf(X)


In [ ]:
# Pipelineに使う独自関数
class MyPreprocessor(object,):
    def fit(self, X, y):
        print(" MyPreprocessor.fit")
        return X
    def fit_transform(self, X, y, feature_num, selector_step, model):
        print(" MyPreprocessor.fit_transform")
        return getSelectorFeature(X,y,feature_num, selector_step, model)
    def predict(self, X, train_col_list):
        print(" MyPreprocessor.predict")
        return getAdjustTestColDf(X, train_col_list)

In [ ]:
#### 学習 ####
model_def_list = [
    ExtraTreesClassifier(n_estimators = ESTIMATOR_NUM),
    RandomForestClassifier(n_estimators=ESTIMATOR_NUM, random_state=RANDOM_STATE_VAL),
    DecisionTreeClassifier(),

#     LogisticRegression(penalty='l2', C=1.5, random_state=RANDOM_STATE_VAL,max_iter=120,
#                        solver="liblinear"),
#     LogisticRegression(penalty='l2', C=1.5, random_state=RANDOM_STATE_VAL,max_iter=120,
#                        solver="newton-cg"),
#     LogisticRegression(penalty='l2', C=1.5, random_state=RANDOM_STATE_VAL,max_iter=1200,
#                        solver="newton-cg"),
#     LogisticRegression(penalty='l2', C=1.0, random_state=RANDOM_STATE_VAL,max_iter=2000,
#                        solver="liblinear"),
]

test_score_list = []
model_fit_list = []
select_train_col_list = []

pipeline = Pipeline([
#     ('MyStandardScaler', MyStandardScaler()),   #標準化
     ('MyPreprocessor', MyPreprocessor()),   #独自関数
])

for model in model_def_list:
    start = time.time()
    print("==================================================")
    print(model)
    
    X_train = pipeline.fit_transform(act_train_Df_2, train_target_Ser,
                                     MyPreprocessor__feature_num = FEATURE_NUM,
                                     MyPreprocessor__selector_step = SELECTOR_STEP,
                                     MyPreprocessor__model = model,
                                    )
    select_train_col_list.append(X_train.columns.tolist())
#    X_train = act_train_Df_2
    
    model.fit(X_train, train_target_Ser.values.ravel())
    model_fit_list.append(model)
    
    cv_results = cross_validate(model,
                                                 act_train_Df_2,
                                                 train_target_Ser.values.ravel(),      #1次元のARRAY
                                                 cv=CV_VAL,
                                                 scoring='accuracy')
    print("test_score = ", np.mean(cv_results["test_score"]))
    test_score_list.append(np.mean(cv_results["test_score"]))

    elapsed_time = time.time() - start
    print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")
    

print("----------------------------------------------------------------------------------------")
# test_score が最大のアルゴリズムを適用する
print(test_score_list)
selected_index = np.argmax(test_score_list)
selected_model = model_fit_list[selected_index]
print("selected_model : {}\n{}".format(selected_model, test_score_list[selected_index]))

In [ ]:
#### 予測 ####

X_test = pipeline.predict(act_test_Df_2, train_col_list = select_train_col_list[selected_index])
#X_test = act_test_Df_2

#test_pred = selected_model.predict(X_test)
test_pred = selected_model.predict_proba(X_test)[:,1]
target_Df = pd.DataFrame(
    {
        target_col_name: test_pred 
    })

In [ ]:
output_Df = test_id_Df.join(target_Df)

In [ ]:
dispalyDf(output_Df, True, 10, "output_Df")

In [ ]:
output_Df[target_col_name].unique()[:10]

In [ ]:
# ディレクトリなければ作る
output_dir_path = os.path.join(os.getcwd(), 'output')
print(output_dir_path)
if not os.path.exists(output_dir_path):
    os.makedirs(output_dir_path) 

In [ ]:
# 出力
output_Df.to_csv('output/submission.csv',
                 sep=",",
                 header=True,
                 index=False,
                 mode="w",
                 encoding="utf-8",
                 line_terminator="\n"
)

In [ ]:
########################################################

In [ ]:
raise Exception("END♪")

In [ ]:
########################################################